# 3. Building Prediction Model

### Tensorflow

In [18]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

In [19]:
X_train = np.load('X_train.npy')
X_test = np.load('X_test.npy')
y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')
X_test_input = np.load('X_test_input.npy')

In [20]:
display('shape of X :', np.shape(X_test),'shape of y :', np.shape(y_test))

'shape of X :'

(2431135, 78)

'shape of y :'

(2431135,)

In [21]:
y_test = np.reshape(y_test, (-1, 1))
y_train = np.reshape(y_train, (-1, 1))

In [ ]:
np.nan_to_num(X_train, copy=False)
np.nan_to_num(X_test, copy=False)

array([[  2.56000000e+02,   0.00000000e+00,   0.00000000e+00, ...,
          2.51712000e+05,   0.00000000e+00,   0.00000000e+00],
       [  1.56800000e+03,   0.00000000e+00,   0.00000000e+00, ...,
          2.62478000e+05,   0.00000000e+00,   0.00000000e+00],
       [  7.17000000e+02,   0.00000000e+00,   0.00000000e+00, ...,
          2.32896000e+05,   0.00000000e+00,   0.00000000e+00],
       ..., 
       [  1.72000000e+02,   0.00000000e+00,   0.00000000e+00, ...,
          2.09737000e+05,   0.00000000e+00,   0.00000000e+00],
       [  5.56000000e+02,   0.00000000e+00,   0.00000000e+00, ...,
          2.60876000e+05,   0.00000000e+00,   0.00000000e+00],
       [  5.89000000e+02,   0.00000000e+00,   0.00000000e+00, ...,
          2.87555000e+05,   0.00000000e+00,   0.00000000e+00]])

In [ ]:
scaler  = MinMaxScaler()
scaler.fit(X_train)
X_train = X_train[~np.isnan(X_train).any(axis=1)]
X_train = scaler.transform(X_train)

X_test = X_test[~np.isnan(X_test).any(axis=1)]
X_test = scaler.transform(X_test)

In [ ]:
tf.reset_default_graph()

In [ ]:
learning_rate = 0.01


X = tf.placeholder(tf.float32, [None, 78], name = 'x-input')
Y = tf.placeholder(tf.float32, [None, 1], name = 'y-input')

keep_prob = tf.placeholder(tf.float32)


with tf.name_scope("layer1") as scope:
    
    W1 = tf.get_variable("weight1", shape=[78, 50],
                     initializer=tf.contrib.layers.xavier_initializer())
    b1 = tf.Variable(tf.random_normal([50]), name='bias1')
    layer1 = tf.nn.relu(tf.matmul(X, W1) + b1)
    
    w1_hist = tf.summary.histogram("weights1", W1)
    b1_hist = tf.summary.histogram("biases1", b1)
    layer1_hist = tf.summary.histogram("layer1", layer1)
    
    
    tf.add_to_collection('vars', W1)

    layer1 = tf.nn.dropout(layer1, keep_prob=keep_prob)


with tf.name_scope("layer2") as scope:

    W2 = tf.get_variable("weight2", shape = [50, 25],
                     initializer=tf.contrib.layers.xavier_initializer())
    b2 = tf.Variable(tf.random_normal([25]), name='bias2')
    layer2 = tf.nn.relu(tf.matmul(layer1, W2) + b2)
    
    
    tf.add_to_collection('vars', W2)

    w2_hist = tf.summary.histogram("weights2", W2)
    b2_hist = tf.summary.histogram("biases2", b2)
    layer2_hist = tf.summary.histogram("layer2", layer2)
    
    layer2 = tf.nn.dropout(layer2, keep_prob=keep_prob)

    
with tf.name_scope("layer3") as scope:
    W3 = tf.get_variable("weight3", shape = [25, 10],
                     initializer=tf.contrib.layers.xavier_initializer())
    b3 = tf.Variable(tf.random_normal([10]), name='bias3')
    layer3 = tf.nn.relu(tf.matmul(layer2, W3) + b3)
    

    tf.add_to_collection('vars', W3)

    w3_hist = tf.summary.histogram("weights3", W3)
    b3_hist = tf.summary.histogram("biases3", b3)
    layer3_hist = tf.summary.histogram("layer3", layer3)
    
    layer3 = tf.nn.dropout(layer3, keep_prob=keep_prob)
    
with tf.name_scope("layer4") as scope:
    W4 = tf.get_variable("weight4", shape = [10, 1],
                     initializer=tf.contrib.layers.xavier_initializer())
    b4 = tf.Variable(tf.random_normal([1]), name='bias4')
    hypothesis = tf.matmul(layer3, W4) + b4
    
    tf.add_to_collection('vars', W4)

    w4_hist = tf.summary.histogram("weights4", W4)
    b4_hist = tf.summary.histogram("biases4", b4)
    hypothesis_hist = tf.summary.histogram("hypothesis", hypothesis)


    
    
# cost/loss function
with tf.name_scope("cost") as scope:
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
    cost_summ = tf.summary.scalar("cost", cost)

with tf.name_scope("train") as scope:
    train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)



# Launch graph
with tf.Session() as sess:
    # tensorboard --logdir=./logs/xor_logs
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./logs/recommendation")
    writer.add_graph(sess.graph)  # Show the graph

    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in range(10001):
        summary, _ = sess.run([merged_summary, train], feed_dict={X: X_train, Y: y_train,  keep_prob: 0.7})
        writer.add_summary(summary, global_step=global_step)
        global_step = 1
        global_step +=10  

        if step % 100 == 0:
            print(step, sess.run(cost, feed_dict={
                  X: X_train, Y: y_train}), sess.run([W1, W2, W3, W4]))


print('Learning Finished!')

correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: X_test, Y: y_test,  keep_prob: 1.0}))

#### Tensorboard

In [ ]:
tensorboard --logdir=./logs

#### Save Trained Tensorflow Model

In [ ]:
saver = tf.train.Saver()
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver.save(sess, 'trained_model')

#### Run Trained Tensorflow Model

In [ ]:
sess = tf.Session()
new_saver = tf.train.import_meta_graph('trained_model.meta')
new_saver.restore(sess, tf.train.latest_checkpoint('./'))
all_vars = tf.get_collection('vars')
for v in all_vars:
    v_ = sess.run(v)
    print(v_)

#### Dealing with Large Data

In [ ]:
filename_queue = tf.train.string_input_producer([...])
reader = tf.TextLineReader()
_, line = reader.read(filename_queue)

line = tf.decode_csv(line, record_defaults=default)
label_batch, feature_batch = tf.train.shuffle_batch([label, feature], batch_size=batch_size, capacity=512, min_after_dequeue=256,  num_threads=8)